In [34]:
# Carga de librerias
# ------------------
# Librerias de uso general
import holidays
from google.cloud import storage

# Manejo de datos
import numpy as np
import pandas as pd

import kagglehub
import shutil
import os

from sklearn.feature_extraction import DictVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import os
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
import pickle
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.metrics import f1_score, mean_squared_error
from math import sqrt

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Estadística y series temporales
from statsmodels.tsa.seasonal import seasonal_decompose

# Se importan las funciones
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [35]:
column_mapper = {
    'Service': 'fecha',
    'Relational Database Service($)': 'rds',
    'EC2-Instances($)': 'ec2',
    'FSx($)': 'fsx',
    'Elastic File System($)': 'efs',
    'EC2-Other($)': 'ec2_other',
    'CloudWatch($)': 'cloudwatch',
    'Elastic Load Balancing($)': 'elb',
    'S3($)': 's3',
    'Backup($)': 'backup',
    'Key Management Service($)': 'kms',
    'DataSync($)': 'data_sync',
    'Secrets Manager($)': 'secrets_manager',
    'Resilience Hub($)': 'resiliency',
    'Total costs($)': 'total_costs'
}

Carga de los Datos
Se hace la carga de los datos desde la ruta Data 

In [36]:
# Datos con corte a Abril usar: "costs.csv"

df = pd.read_csv('/Users/angeleduardogamarrarios/Repositorio_UDEM/MLops_AMICO/data/costs.csv')
df.head()

,Service,Relational Database Service($),EC2-Instances($),FSx($),Elastic File System($),EC2-Other($),CloudWatch($),S3($),Elastic Load Balancing($),Backup($),Key Management Service($),DataSync($),Secrets Manager($),Resilience Hub($),Total costs($)
0,Service total,34131.482733,23531.788153,5152.073356,2830.548352,2132.939335,1543.902136,778.604880,758.963353,530.170434,91.476443,34.06381,0.178930,0.0,71516.191916
1,2024-06-01,31.851525,4.612654,12.472440,1.789663,0.611443,0.036701,1.156349,2.160000,0.281071,0.234161,NaN,0.000255,0.0,55.206262
2,2024-06-02,61.039787,NaN,12.460864,1.789663,0.322181,0.016653,1.126354,2.160000,0.281071,0.235058,NaN,0.000295,NaN,79.431926
3,2024-06-03,71.416393,63.161250,12.654853,1.790544,5.613880,0.429317,1.119325,2.160042,0.281071,0.240077,NaN,0.000225,NaN,158.866978
4,2024-06-04,54.833184,68.402379,12.658942,1.791199,5.682819,0.459109,1.116106,2.160139,0.281314,0.238019,NaN,0.000225,NaN,147.623435


In [37]:
# Mapeo de columnas 
df.rename(columns=column_mapper, inplace=True)
# Elimina la fila con índice 0 y ka columna total_cost
df = df.drop(0, axis=0)
df = df.drop('total_costs', axis=1)

# Prepocesar datos

Para nuestro caso de analisis vamos a aplicar una normalizacion

In [38]:
#Separar las variables continuas de las demás
df_continuas=df[["rds",
                       "ec2",
                       "fsx",
                       "efs",
                       "ec2_other",
                       "cloudwatch",
                       "s3",
                       "elb",
                       "backup",
                       "kms",
                       "data_sync",
                       "secrets_manager",
                       "resiliency"]]

In [39]:
# EL conjunto de datos se divide en una proporción de 60% (Entrenamiento), 20% (Validación) y 20% (Prueba).
X_train_val, X_test = train_test_split(df_continuas, test_size=0.2, random_state=42)
X_train, X_val = train_test_split(X_train_val, test_size=0.25, random_state=42) # 0.25 * 0.80 = 0.20 (20% para Validation)

In [40]:
# Normalizar los datos 
preprocessor_pipeline = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor_pipeline.fit(X_train)

X_train_scaled = preprocessor_pipeline.transform(X_train)
X_val_scaled = preprocessor_pipeline.transform(X_val)
X_test_scaled = preprocessor_pipeline.transform(X_test)

# 4. Verificación y Persistencia (MLOps)
print(f"Media de 'rds' en X_train original: {X_train['rds'].mean():.4f}")
print(f"Media de 'rds' en X_train escalado: {X_train_scaled[:, X_train.columns.get_loc('rds')].mean():.4f}") # La media debe ser cercana a 0

# Guardar el Pipeline (¡Esto es clave para el MLOps!)
# Debes guardar el objeto preprocessor_pipeline para usarlo en producción
# cuando llegue un nuevo dato de vino.

OUTPUT_DIR = "./data/"
os.makedirs(OUTPUT_DIR, exist_ok=True)

with open(os.path.join(OUTPUT_DIR, "preprocessor_pipeline.pkl"), "wb") as f:
    pickle.dump(preprocessor_pipeline, f)

print("-" * 50)
print(f"✅ Pipeline y datos escalados listos. El preprocesador se guardó en: {OUTPUT_DIR}")


Media de 'rds' en X_train original: 96.0687
Media de 'rds' en X_train escalado: -0.0000
--------------------------------------------------
✅ Pipeline y datos escalados listos. El preprocesador se guardó en: ./data/


In [42]:
# Crear los DF de nuevo con los datos ecaldos entregado en un array
# 1. Crear el DataFrame escalado
X_train_scaled_df = pd.DataFrame(
    # 1. Los datos son el array de NumPy
    X_train_scaled, 
    
    # 2. Los índices (filas) se toman del DataFrame original
    index=X_train.index, 
    
    # 3. Los nombres de las columnas se toman del DataFrame original
    columns=X_train.columns 
)

# 2. Repetir para los otros conjuntos
X_val_scaled_df = pd.DataFrame(
    X_val_scaled, 
    index=X_val.index, 
    columns=X_val.columns
)

X_test_scaled_df = pd.DataFrame(
    X_test_scaled, 
    index=X_test.index, 
    columns=X_test.columns
)

# 4. Verificación
print("\n--- Verificación del DataFrame escalado ---")
print(f"Tipo de X_train_scaled_df: {type(X_train_scaled_df)}")
print(f"Las primeras 5 filas de X_train_scaled_df:\n{X_train_scaled_df.head()}")
print(f"Media de 'rds' en X_train escalado (DF): {X_train_scaled_df['rds'].mean():.4f}")


--- Verificación del DataFrame escalado ---
Tipo de X_train_scaled_df: <class 'pandas.core.frame.DataFrame'>
Las primeras 5 filas de X_train_scaled_df:
          rds       ec2       fsx       efs  ec2_other  cloudwatch        s3  \
175 -0.055722 -0.455170  0.662202 -0.666575   0.129742   -0.405917 -0.247247   
233 -0.069585  1.030785  0.492330  0.707702   0.541876   -0.471159  0.564653   
351  3.891164 -0.703529  0.508479  1.861979  -1.265586   -0.166167  1.434175   
75  -0.648316 -0.494546  0.333014 -0.940589   0.849970   -0.330212 -0.945479   
66  -0.665479 -0.702253 -1.098427 -0.944476  -0.040083   -0.472826 -1.103862   

          elb    backup       kms  data_sync  secrets_manager  resiliency  
175 -0.408174 -0.575719  0.224661  -0.111943        -0.539126         0.0  
233 -0.459486  0.489771 -0.058055  -0.111944        -0.140724         0.0  
351 -0.116118 -0.381946  2.932068  -0.111943        -0.695362         0.0  
75  -0.328749 -0.995102 -1.013494  -0.111943         0.062383 

Isolation Forest (El Enfoque Robusto)
El Isolation Forest es rápido, escalable y muy robusto para datos multivariados con muchos outliers.

In [52]:
# Creamos una copia del DF de train original para poder ser usado en otros modelos 
X_train_scaled_df_is = X_train_scaled_df.copy() 
# creamos una columna de etiquetas verdaderas *para el ejemplo*
# En la vida real, necesitarías cargar estas etiquetas.
# Ejemplo de etiquetas verdaderas simuladas:
# 99% normales (1), 1% anómalas (-1)
X_train_size = X_train_scaled_df_is.shape[0]
n_anomalies = int(X_train_size * 0.01)
y_true = np.ones(X_train_size)
y_true[:n_anomalies] = -1 # Los primeros datos son anómalos
np.random.shuffle(y_true) # Mezclamos las etiquetas
X_train_scaled_df_is['is_anomaly_true'] = y_true # Agregamos la columna de etiquetas verdaderas


# Inicializar y entrenar el modelo
iso_forest = IsolationForest(
    n_estimators=100,
    contamination=contamination_factor,
    random_state=42,
    n_jobs=-1
)

X_train_features = X_train_scaled_df_is.copy() 

# Entrenar el modelo (No supervisado: no necesita etiquetas)
iso_forest.fit(X_train_features.drop(columns=['is_anomaly_true'])) # Entrenamos sin la etiqueta verdadera

# Predecir las anomalías: -1 para anomalía, 1 para dato normal
X_train_scaled_df_is['anomaly_iso_forest'] = iso_forest.predict(X_train_features.drop(columns=['is_anomaly_true']))

# Obtener la 'Puntuación de Anomalía'
X_train_scaled_df_is['score_iso_forest'] = iso_forest.decision_function(X_train_features.drop(columns=['is_anomaly_true']))

# Mostrar los días anómalos
print("\n--- Anomalías detectadas por Isolation Forest (ejemplo) ---")
anomalies_if = X_train_scaled_df_is[X_train_scaled_df_is['anomaly_iso_forest'] == -1]
print(f"Total de anomalías detectadas: {len(anomalies_if)}")
print(anomalies_if.head())
# Nota: El umbral para la detección es ajustado por el 'contamination' que elegiste.

# --- 2. CÁLCULO DE MÉTRICAS DE DESEMPEÑO ---

# a) Métricas de Clasificación (F1-Score)
# Necesitan las etiquetas verdaderas (y_true) y las etiquetas predichas (y_pred)

y_pred = X_train_scaled_df_is['anomaly_iso_forest'].values
y_true = X_train_scaled_df_is['is_anomaly_true'].values

# El F1-Score es la media armónica entre precisión y exhaustividad.
# Es ideal cuando las clases están desbalanceadas (muchos normales, pocos anómalos).
f1 = f1_score(y_true, y_pred, average='binary', pos_label=-1) 

print(f"F1-Score (enfocado en anomalías): {f1:.4f}")

# b) Métrica de Error (RMSE)
# El RMSE se usa típicamente para modelos de Regresión, pero se puede usar para medir
# qué tan lejos están las *puntuaciones* de anomalía de un valor objetivo.

# **ADVERTENCIA sobre RMSE en Detección de Anomalías:**
# El RMSE no es una métrica estándar para evaluar el rendimiento de clasificación
# de Isolation Forest. No obstante, si se quiere forzar la métrica:

# Creamos una 'puntuación ideal' donde los valores normales tienen 1.0 y los anómalos tienen un valor bajo (ej. 0.0)
# Para el ejemplo, usaremos la puntuación de decisión (score_iso_forest)
# La forma correcta de usar RMSE aquí es compleja, así que mostraremos cómo calcularlo
# si tu objetivo fuese predecir la puntuación verdadera.

# Para simplificar y mostrar el cálculo:
# Asumamos que tu objetivo (y_true) son las etiquetas numéricas, y queremos ver
# el "error" de la puntuación de decisión.
# (Esto no es un uso estándar, pero cumple con el requisito de mostrar el cálculo)
# Vamos a simular un 'score' objetivo: 1 para normal, 0 para anomalía
y_target_score = np.where(y_true == 1, 1.0, 0.0) 
y_pred_score = X_train_scaled_df_is['score_iso_forest'].values # Las puntuaciones de decisión

rmse_score = sqrt(mean_squared_error(y_target_score, y_pred_score))

print(f"RMSE (entre puntuación ideal y decisión): {rmse_score:.4f}")


# --- 3. RESUMEN DE MÉTRICAS ---

# Puedes agrupar las métricas en un diccionario para un informe fácil
performance_metrics = {
    'F1_Score': f1,
    'RMSE_Score': rmse_score,
    # Otras métricas comunes en anomalías:
    # 'Precision': precision_score(y_true, y_pred, pos_label=-1),
    # 'Recall': recall_score(y_true, y_pred, pos_label=-1)
}

print("\n--- RESUMEN DE DESEMPEÑO ---")
for metric, value in performance_metrics.items():
    print(f"{metric}: {value:.4f}")


--- Anomalías detectadas por Isolation Forest (ejemplo) ---
Total de anomalías detectadas: 3
          rds       ec2       fsx       efs  ec2_other  cloudwatch        s3  \
351  3.891164 -0.703529  0.508479  1.861979  -1.265586   -0.166167  1.434175   
349  3.902792  0.535344  0.772158  1.861837   0.704645    1.506159  1.424562   
344  3.171395 -0.539961  0.593477  1.861602  -0.722283   -0.178437  1.426032   

          elb    backup       kms  data_sync  secrets_manager  resiliency  \
351 -0.116118 -0.381946  2.932068  -0.111943        -0.695362         0.0   
349  6.726628  2.150503  2.939841  -0.111943         8.569433         0.0   
344  3.981615  2.150536  2.931883  -0.111943        -0.539126         0.0   

     is_anomaly_true  anomaly_iso_forest  score_iso_forest  
351              1.0                  -1         -0.044952  
349              1.0                  -1         -0.060478  
344              1.0                  -1         -0.005552  
F1-Score (enfocado en anomalías)

One-Class SVM (OC-SVM)
OC-SVM intenta encontrar un hiperplano que separe la mayoría de los datos "normales" del origen, marcando los puntos que quedan fuera como anomalías.

In [54]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM  # Importación para OC-SVM
from sklearn.neighbors import LocalOutlierFactor # Importación para LOF

# Creamos una copia del DF de train original para poder ser usado en otros modelos 
X_train_scaled_df_oc = X_train_scaled_df.copy() 

# --- CONFIGURACIÓN DEL MODELO ---

# El parámetro 'nu' es el equivalente al 'contamination' factor,
# representando la fracción superior de anomalías esperadas.
# 'gamma' define la influencia de un punto de entrenamiento individual
# (usamos 'auto' para que se calibre por 1/n_features).
contamination_factor = 0.01  # Esperamos un 1% de anomalías

# Inicializar el modelo OC-SVM
oc_svm = OneClassSVM(
    kernel='rbf',  # Kernel de función de base radial, común en anomalías
    gamma='auto',
    nu=contamination_factor
)

X_train_features = X_train_scaled_df_is.copy() 

# Entrenar el modelo
oc_svm.fit(X_train_features)

# Predecir las anomalías: -1 para anomalía, 1 para dato normal
X_train_scaled_df_oc['anomaly_oc_svm'] = oc_svm.predict(X_train_features)

# Obtener la 'Puntuación de Anomalía' (cuanto menor, más anómalo)
X_train_scaled_df_oc['score_oc_svm'] = oc_svm.decision_function(X_train_features)

# Mostrar los días anómalos
print("\n--- Anomalías detectadas por One-Class SVM ---")
anomalies_ocsvm = X_train_scaled_df_oc[X_train_scaled_df_oc['anomaly_oc_svm'] == -1]
print(f"Total de anomalías detectadas: {len(anomalies_ocsvm)}")
print(anomalies_ocsvm.sort_values(by='score_oc_svm').head())


--- Anomalías detectadas por One-Class SVM ---
Total de anomalías detectadas: 13
          rds       ec2       fsx       efs  ec2_other  cloudwatch        s3  \
351  3.891164 -0.703529  0.508479  1.861979  -1.265586   -0.166167  1.434175   
262  0.139671  1.033179  1.427751  1.065621   0.089126   -0.418847  1.178391   
323  1.836257  0.893155  1.078494  1.936324   1.628101    2.352258  1.486355   
319  1.327360  1.053907  1.066812  1.783952   1.587446    2.344113  1.592530   
39   0.091915  0.316006 -1.169444 -0.958107   0.282216   -0.454032 -1.393626   

          elb    backup       kms  data_sync  secrets_manager  resiliency  \
351 -0.116118 -0.381946  2.932068  -0.111943        -0.695362         0.0   
262 -0.219831  0.808493  0.825572  -0.107682         2.460605         0.0   
323 -0.457923  1.644362  2.269787  -0.111943        -0.226654         0.0   
319  0.374527  1.466923  2.277282  -0.111943         2.093451         0.0   
39   0.096049 -1.003566 -0.909490  -0.111943        

Local Outlier Factor (LOF) LOF es un modelo basado en la densidad. Mide cuán aislado está un punto con respecto a sus vecinos más cercanos. Un score LOF mucho mayor que 1 indica una anomalía.

In [56]:
# Creamos una copia del DF de train original para poder ser usado en otros modelos 
X_train_scaled_df_lof = X_train_scaled_df.copy() 

# --- CONFIGURACIÓN DEL MODELO ---

# 'n_neighbors': Número de vecinos a considerar (típicamente entre 20 y 100).
# 'contamination': La proporción esperada de anomalías (similar a Isolation Forest).
n_neighbors = 20
contamination_factor = 0.01

# Inicializar el modelo LOF
# Nota: LOF se inicializa en 'novelty=False' para fines de predicción.
lof = LocalOutlierFactor(
    n_neighbors=n_neighbors,
    contamination=contamination_factor,
    novelty=False  # Para usarlo en modo detección de outliers en el conjunto de entrenamiento
)

# Ajustar y predecir (LOF realiza el ajuste y la predicción en un solo paso)
X_train_scaled_df_lof['anomaly_lof'] = lof.fit_predict(X_train_scaled_df_lof)

# La 'Puntuación de Anomalía' de LOF es el factor LOF (cuanto mayor, más anómalo)
# Nota: LOF devuelve el Factor de Anormalidad Local Negativo. Lo negamos para que
# los valores más altos sigan significando mayor anomalía, como es habitual.
X_train_scaled_df_lof['score_lof'] = -lof.negative_outlier_factor_

# Mostrar los días anómalos
print("\n--- Anomalías detectadas por Local Outlier Factor (LOF) ---")
anomalies_lof = X_train_scaled_df_lof[X_train_scaled_df_lof['anomaly_lof'] == -1]
print(f"Total de anomalías detectadas: {len(anomalies_lof)}")
print(anomalies_lof.sort_values(by='score_lof', ascending=False).head())


--- Anomalías detectadas por Local Outlier Factor (LOF) ---
Total de anomalías detectadas: 3
          rds       ec2       fsx       efs  ec2_other  cloudwatch        s3  \
203 -0.328424  0.436305  0.450120 -0.129187   0.464148   -0.462137 -0.085947   
224 -0.250073  1.075655  0.459695  0.194370   0.590359   -0.474813  0.177261   
252  0.036355  1.020134  1.813887  0.904283   0.227462   -0.380775  1.060813   

          elb    backup       kms  data_sync  secrets_manager  resiliency  \
203  0.263859 -0.138673 -0.053058   9.541929        -0.671926         0.0   
224 -0.045358  0.028645  0.061032  -0.111942         8.460067         0.0   
252  0.256140  2.096955  0.797905  10.679210        -0.086041         0.0   

     is_anomaly_true  anomaly_iso_forest  score_iso_forest  anomaly_lof  \
203              1.0                   1          0.132619           -1   
224              1.0                   1          0.125949           -1   
252              1.0                   1          0